In [1]:
import warnings
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder
%run models.ipynb

warnings.filterwarnings('ignore')
random.seed(100)

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [36]:
dataset = "re3d"

In [37]:
features = pd.read_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Features/Latest/features_"+dataset+".csv", index_col=0)
# features.head(1)

In [38]:
features["rel_type"].unique()

array(['CoLocated', 'CommWith', 'FightingAgainst', 'BelongsTo'],
      dtype=object)

In [39]:
columns = ['sentence_length', 'root_word', 'root_index',
       'entity_1_index', 'entity_2_index', 'entity_distance',
       'no_words_before_entity_1', 'no_words_after_entity_2',
       'entity_1_root_distance', 'entity_2_root_distance', 'entity 1 name',
       'entity 2 name', 'entity_pos_1',
       'entity_pos_2', 'entity_dep_1', 'entity_dep_2', 'shortest_distance', 'rel_type']
features = features[columns]
features.rename(columns={"rel_type": "label"}, inplace=True)

In [40]:
features

,sentence_length,root_word,root_index,entity_1_index,entity_2_index,entity_distance,no_words_before_entity_1,no_words_after_entity_2,entity_1_root_distance,entity_2_root_distance,entity 1 name,entity 2 name,entity_pos_1,entity_pos_2,entity_dep_1,entity_dep_2,shortest_distance,label
21,23,shows,9,7,9,2,6,13,2,0,Daesh,Marea,PROPN,PROPN,pobj,pobj,2,CoLocated
27,31,inflicting,10,20,25,5,19,5,-10,15,other towns and areas,Syria,ADJ-NOUN-CCONJ-NOUN,PROPN,amod-conj-cc-conj,pobj,6,CoLocated
2,37,met,12,1,1,0,0,35,11,-11,U.S. Army General Joseph Votel,U.S. Air Force civilian employees,PROPN-PROPN-PROPN-PROPN-PROPN,PROPN-PROPN-PROPN-ADJ-NOUN,compound-compound-compound-compound-nsubj,compound-compound-nmod-amod-pobj,0,CommWith
0,24,shown,7,3,7,4,2,16,4,0,U.S. service members,Syria,PROPN-NOUN-NOUN,PROPN,compound-compound-pobj,pobj,4,CoLocated
5,40,advising,6,4,10,6,3,29,2,4,U.S. forces,Syrian Democratic Forces,PROPN-NOUN,PROPN-PROPN-PROPN,compound-nsubj,amod-compound-dobj,5,CommWith
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,24,symbolises,1,1,8,7,0,15,0,7,Karim,the region,PROPN,DET-NOUN,nsubj,det-pobj,4,CoLocated
69,12,met,3,1,3,2,0,8,2,0,Karim,Sadiq,PROPN,PROPN,nsubj,conj,2,CoLocated
70,12,met,3,1,3,2,0,8,2,0,Karim,Sadiq,PROPN,PROPN,nsubj,conj,2,CoLocated
71,12,met,3,1,3,2,0,8,2,0,Karim,Sadiq,PROPN,PROPN,nsubj,conj,2,CommWith


In [41]:
# features_norel = features[features["label"]!="no_relation"]
# norel = features[features["label"]=="no_relation"]
# features = features_norel.append(norel.sample(2000))

X = features.iloc[:,:-1]
X = X.fillna("")
y_original =  features.iloc[:,-1]

x_train_original, x_test_original, y_train_original, y_test_original = train_test_split(X, y_original, test_size=0.2)
onehot_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
x_train = onehot_encoder.fit_transform(x_train_original)
x_test = onehot_encoder.transform(x_test_original)

le = LabelEncoder()
y_train = le.fit_transform(y_train_original)
y_test = le.transform(y_test_original)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping

{'BelongsTo': 0, 'CoLocated': 1, 'CommWith': 2, 'FightingAgainst': 3}

In [44]:
unique, count = np.unique(y_original, return_counts = True)
dict(zip(unique, count))

{'Based-In': 664,
 'Business': 278,
 'Citizen-or-Resident': 536,
 'Employ-Executive': 878,
 'Employ-Staff': 870,
 'Employ-Undetermined': 172,
 'Ethnic': 102,
 'Family': 42,
 'Ideology': 72,
 'Located': 636,
 'Member-of-Group': 286,
 'Near': 178,
 'Other': 398,
 'Part-Whole': 534,
 'Subsidiary': 484,
 'no_relation': 2000}

In [24]:
pred = logistic_regression_multiple_classes(x_train, x_test, y_train, y_test, "et")
x_test_log_pred = x_test_original
x_test_log_pred["label"] = y_test
x_test_log_pred["prediction"] = pred
x_test_log_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_logisitic_regression_prediction.csv")

--------Logistic Regression-----------


In [25]:
pred = random_forest_multiple(x_train, x_test, y_train, y_test,"et")
x_test_rf_pred = x_test_original
x_test_rf_pred["label"] = y_test
x_test_rf_pred["prediction"] = pred
x_test_rf_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_random_forest_prediction.csv")


--------Random Forest-----------


In [26]:
pred = svm_multiple("linear", x_train, x_test, y_train, y_test, "et")
x_test_svm_pred = x_test_original
x_test_svm_pred["label"] = y_test
x_test_svm_pred["prediction"] = pred
x_test_svm_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_prediction.csv")

--------Support Vector Classifier linear-----------


In [27]:
pred = svm_multiple("rbf", x_train, x_test, y_train, y_test, "et")
x_test_svmrbf_pred = x_test_original
x_test_svmrbf_pred["label"] = y_test
x_test_svmrbf_pred["prediction"] = pred
x_test_svmrbf_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_rbf_prediction.csv")

--------Support Vector Classifier rbf-----------


In [28]:
pred = svm_multiple("poly", x_train, x_test, y_train, y_test, "et")
x_test_svmpoly_pred = x_test_original
x_test_svmpoly_pred["label"] = y_test
x_test_svmpoly_pred["prediction"] = pred
x_test_svmpoly_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_poly_prediction.csv")


--------Support Vector Classifier poly-----------


In [29]:
pred = xgboost_multiple_class(x_train, x_test, y_train, y_test, "et")
x_test_xg_pred = x_test_original
x_test_xg_pred["label"] = y_test
x_test_xg_pred["prediction"] = pred
x_test_xg_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_xgboost_prediction.csv")


In [30]:
pred = adaboost_multiple(x_train, x_test, y_train, y_test, "et")
x_test_ada_pred = x_test_original
x_test_ada_pred["label"] = y_test
x_test_ada_pred["prediction"] = pred
x_test_ada_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_adaboost_prediction.csv")
